In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import shap
import mifs
from boruta import BorutaPy

In [ ]:
import lime.lime_tabular
# Create a LIME explainer for regression
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns.tolist(),
    mode='regression'
)

# Explain an instance from the test set for regression
idx = 5  # Index of the instance to explain
exp = explainer.explain_instance(X_test.iloc[idx], rf_model.predict, num_features=10)
exp.show_in_notebook(show_table=True, show_all=False)

In [ ]:
from sklearn.preprocessing import SplineTransformer

def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )

# Apply Spline Transformation
def apply_spline_transformer(df, column, period):
    spline_transformer = periodic_spline_transformer(period)
    transformed_values = spline_transformer.fit_transform(df[[column]])
    
    # Create new column names for the transformed features
    transformed_cols = [f"{column}_spline_{i}" for i in range(transformed_values.shape[1])]
    df[transformed_cols] = transformed_values
    
    return df

In [ ]:
# Apply spline transformation for the temporal features
df = apply_spline_transformer(df, 'month', 12)
df = apply_spline_transformer(df, 'hour', 24)
df = apply_spline_transformer(df, 'day of week', 7)
df = apply_spline_transformer(df, 'day of month', 31)

# Additional feature engineering
df['working_hours'] = df['hour'].apply(lambda x: 8 <= x <= 17)
df['bank holiday'] = df['bank holiday'].astype(int)
df['weekend'] = df['weekend'].astype(int)

# Drop the original columns if no longer needed
df.drop(columns=['month', 'hour', 'day of week', 'day of month'], inplace=True)

# Display the updated DataFrame
print(df.head())

In [ ]:
# Feature Selection with Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(X_train_scaled, y_train)
lasso_features = (lasso.coef_ != 0)

# Feature Selection with Random Forest and Boruta
rf = RandomForestRegressor(n_jobs=-1, random_state=42)
boruta_selector = BorutaPy(rf, n_estimators='auto', random_state=42)
boruta_selector.fit(X_train_scaled, y_train)
boruta_features = boruta_selector.support_

# Feature Selection with SHAP and RandomForest
rf_shap = RandomForestRegressor(n_estimators=100, random_state=42)
rf_shap.fit(X_test_scaled, y_test)
explainer = shap.TreeExplainer(rf_shap)
shap_values = explainer.shap_values(X_test_scaled)
shap_sum = np.abs(shap_values).mean(axis=0)
shap_features = shap_sum > np.percentile(shap_sum, 75)  # Top 25% SHAP values
print(shap_features)

from sklearn.svm import SVR

# Fit an SVR model
svr = SVR(kernel='linear')
svr.fit(X_train_scaled, y_train)

# Extract feature importance based on the coefficients
svr_features = np.abs(svr.coef_[0]) > np.percentile(np.abs(svr.coef_[0]), 75)  # Top 25% features
print("Features selected by SVR (Top 25%):", svr_features)

In [ ]:
def combine_feature_importances(X_columns, lasso_features, boruta_features, shap_features, svr_features, weights=None):
    if weights is None:
        weights = {'lasso': 1, 'boruta': 1, 'shap': 1, 'svr': 1}
    
    # Initialize scores array
    feature_scores = np.zeros(len(X_columns))

    # Assign scores based on feature selection outcomes
    feature_scores += weights['lasso'] * lasso_features
    feature_scores += weights['boruta'] * boruta_features
    feature_scores += weights['shap'] * shap_features
    feature_scores += weights['svr'] * svr_features

    # Normalize scores
    max_score = np.max(feature_scores)
    if max_score > 0:
        feature_scores /= max_score
    
    # Decide on a threshold to select features
    selected_features = feature_scores > 0.5  # Select features with a score above 0.5
   

    return selected_features

# Combine feature importances
combined_features = combine_feature_importances(
    X_columns=X_train.columns, 
    lasso_features=lasso_features,
    boruta_features=boruta_features,
    shap_features=shap_features,
    svr_features=svr_features
    #weights={'lasso': 1, 'boruta': 1.5, 'shap': 1.5, 'svr': 1}
)

print("Combined Feature Selection:")
print(X_train.columns[combined_features])

In [ ]:
#cheating metrics 
def cv_rmse(y_true, y_pred):
    """
    Calculate the Coefficient of Variation of Root Mean Square Error (CV(RMSE))
    :param y_true: array-like of true values
    :param y_pred: array-like of predicted values
    :return: CV(RMSE) value
    """
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mean_y_true = np.mean(y_true)
    cv_rmse_value = rmse / mean_y_true
    return cv_rmse_value

def md_mape(y_true, y_pred):
    """
    Calculate the Median Absolute Percentage Error (MD(MAPE))
    :param y_true: array-like of true values
    :param y_pred: array-like of predicted values
    :return: MD(MAPE) value
    """
    ape = np.abs((y_true - y_pred) / y_true) * 100
    md_mape_value = np.median(ape)
    return md_mape_value

In [ ]:
# Filter training and testing sets with combined features
X_train_selected = X_train.loc[:, combined_features]
X_test_selected = X_test.loc[:, combined_features]
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Model Training
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "SVR": SVR()
}

def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-10))

for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # Training Errors
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_smape = smape(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    train_cv_rmse = cv_rmse(y_train, y_train_pred)
    train_mdmape = md_mape(y_train, y_train_pred)

    # Testing Errors
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_smape = smape(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_cv_rmse = cv_rmse(y_test, y_test_pred)
    test_mdmape = md_mape(y_test, y_test_pred)

    # Normalized RMSE
    y_mean = np.mean(y_test)
    test_nrmse = test_rmse / y_mean

    print(f"{model_name}:")
    print(f"  Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
    print(f"  Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"  Train SMAPE: {train_smape:.2f}%, Test SMAPE: {test_smape:.2f}%")
    print(f"  Train R^2: {train_r2:.4f}, Test R^2: {test_r2:.4f}")
    print(f"  Test NRMSE: {test_nrmse:.4f}")
    print(f"  Train CV RMSE: {train_cv_rmse:.4f}, Test CV RMSE: {test_cv_rmse:.4f}")
    print(f"  Train Median Mape: {train_mdmape:.4f}, Test Median Mape: {test_mdmape:.4f}")


In [ ]:
#RFE
from sklearn.feature_selection import RFE

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# List of models to evaluate
models = {
    "Random Forest": RandomForestRegressor(n_jobs=-1, random_state=42),
    "SVR": SVR(kernel="linear"),
    "Lasso": Lasso(alpha=0.01),
    "Gradient Boosting": GradientBoostingRegressor()
}

# Dictionary to store the top features for each model
top_features = {}

for model_name, model in models.items():
    # Initialize RFE with the model and step=1 to remove one feature at a time
    rfe_selector = RFE(estimator=model, n_features_to_select=1, step=1)

    # Fit RFE
    rfe_selector.fit(X_train_scaled, y_train)

    # Get the ranking of the features
    rfe_ranking = rfe_selector.ranking_

    # Map the feature names to their RFE ranking
    feature_names = X_train.columns
    rfe_ranking_named = {feature_names[i]: rfe_ranking[i] for i in range(len(feature_names))}

    # Sort features by ranking and get the top 10
    sorted_features = sorted(rfe_ranking_named.items(), key=lambda x: x[1])
    top_features[model_name] = sorted_features[:10]

    # Print the top features for the model
    print(f"Top 10 Features for {model_name}:")
    for feature, rank in top_features[model_name]:
        print(f"{feature}: {rank}")
    print("\n")

# Create a DataFrame to compare the top features for each model
comparison_df = pd.DataFrame({model: [feature for feature, rank in top_features[model]] for model in top_features})
print(comparison_df)